In [1]:
import matplotlib.pyplot as plt

def plot_image_with_corners(image_path, corner_file):
    # Load the image
    image = plt.imread(image_path)

    # Read the corner coordinates from the file
    with open(corner_file, 'r') as file:
        corners = [tuple(map(int, line.strip().split(','))) for line in file]

    # Plot the image and corner points
    fig, ax = plt.subplots()
    ax.imshow(image)

    # Plot the corner points
    for corner in corners:
        x, y = corner
        ax.plot(x, y, 'ro')

    plt.show()
# plot_image_with_corners('padded_image.jpg','corners.txt')

In [2]:
from PIL import Image
import random

def rescale_image(image_path):
    image = Image.open(image_path)
    min_scale = 0.5
    max_scale = 1.2

    # Generate a random scale factor between min_scale and max_scale
    scale_factor = random.uniform(min_scale, max_scale)

    # Calculate the new width and height based on the scale factor
    new_width = int(image.width * scale_factor)
    new_height = int(image.height * scale_factor)

    # Resize the image using the calculated width and height
    resized_image = image.resize((new_width, new_height))

    # Get the coordinates of the corners of the rescaled image
    corners = [(0, 0), (new_width, 0), (new_width, new_height), (0, new_height)]

    # Save the coordinates to a text file
    with open('corners.txt', 'w') as file:
        for corner in corners:
            file.write(f"{corner[0]},{corner[1]}\n")

    return resized_image

In [3]:
from PIL import Image

def add_padding(image):

    # image = Image.open(image)
    maxpad = max(image.size[0], image.size[1])

    top = maxpad // 2
    right = maxpad // 2
    bottom = maxpad // 2
    left = maxpad // 2
    # Calculate the new width and height including padding
    padded_width = image.width + right + left
    padded_height = image.height + top + bottom

    # Create a new blank image with the desired width, height, and color
    padded_image = Image.new(image.mode, (padded_width, padded_height), color='black')

    # Paste the original image onto the padded image with the desired offsets
    padded_image.paste(image, (left, top))

    # Get the coordinates of the corners of the padded image
    corners = [(left, top),
               (left + image.width, top),
               (left + image.width, top + image.height),
               (left, top + image.height)]

    # Save the coordinates to a text file
    with open('corners.txt', 'w') as file:
        for corner in corners:
            file.write(f"{corner[0]},{corner[1]}\n")

    return padded_image



# padded_image = add_padding(image, top_padding, right_padding, bottom_padding, left_padding)


In [4]:
from PIL import Image
import random
import math

def random_rotate(image):
    # image = Image.open(image)
    max_angle=50
    # Generate a random rotation angle between -max_angle and max_angle
    alpha = random.uniform(-max_angle, max_angle)
    # alpha=45


    # Rotate the image by the generated angle
    rotated_image = image.rotate(alpha, expand=True)

    # Get the new width and height of the rotated image
    new_width, new_height = rotated_image.size

    # Get the center coordinates of the rotated image
    center_x = new_width / 2
    center_y = new_height / 2


    theta = math.radians(-alpha)
    rot_matrix = [
        [math.cos(theta), -math.sin(theta)],
        [math.sin(theta), math.cos(theta)]
]

    im_center_a = (image.size[0] / 2, image.size[1] / 2)
    im_center_b = (rotated_image.size[0] / 2, rotated_image.size[1] / 2)


    # Get the original coordinates of the corners of the padded image
    with open('corners.txt', 'r') as file:
        corners = [tuple(map(int, line.strip().split(','))) for line in file]

    # Calculate the updated coordinates of the corners after rotation
    updated_corners = []
    for corner in corners:
        x, y = corner

        rotated_p = (
        rot_matrix[0][0] * (x - im_center_a[0]) + rot_matrix[0][1] * (y - im_center_a[1]) + im_center_b[0],
        rot_matrix[1][0] * (x - im_center_a[0]) + rot_matrix[1][1] * (y - im_center_a[1]) + im_center_b[1]
        )

        updated_x = rotated_p[0]
        updated_y = rotated_p[1]

        updated_corners.append((int(updated_x), int(updated_y)))

    # Save the updated coordinates to a new text file
    with open('corners.txt', 'w') as file:
        for corner in updated_corners:
            file.write(f"{corner[0]},{corner[1]}\n")

    return rotated_image

# Example usage
# image_path = 'padded_image.jpg'

# rotated_image = random_rotate(image_path, max_angle)

# Save the rotated image
# rotated_image.save('rotated_image.jpg')


In [5]:
import cv2
import numpy as np
import random
# Load the image
def perpective(image):
  image = cv2.imread(image)

  # Define the source points (coordinates of a rectangle in the source image)
  source_points = np.float32([[0, 0], [image.shape[1] - 1, 0], [image.shape[1] - 1, image.shape[0] - 1], [0, image.shape[0] - 1]])

  # Define the destination points (coordinates of a quadrilateral in the transformed image)
  destination_points = np.float32([[0, 0], [image.shape[1] - 1, 0], [int(0.6* (image.shape[1] - 1)), image.shape[0] - 1], [int(0.4 * (image.shape[1] - 1)), image.shape[0] - 1]])
  # destination_points = np.float32([[0, 0], [image.shape[1] - 1, 0], [int(random.random()* (image.shape[1] - 1)), image.shape[0] - 1], [int(random.random() * (image.shape[1] - 1)), image.shape[0] - 1]])


  # print(source_points)
  # print(destination_points)
  # Compute the perspective transform matrix
  # print(image.shape)
  # source_points=np.float32([[100,50],[290,50],[100,400],[290,400]])
  # destination_points=np.float32([[130,100],[300,100],[130,450],[300,450]])

  perspective_matrix = cv2.getPerspectiveTransform(source_points, destination_points)

  # Perform the perspective transform on the image
  output_image = cv2.warpPerspective(image, perspective_matrix, (image.shape[1], image.shape[0]))

  # Load the coordinates file
  with open('corners.txt', 'r') as file:
      coordinates = file.read().splitlines()

  # Transform the coordinates using the perspective transform matrix
  transformed_coordinates = []
  for coordinate in coordinates:
      x, y = map(float, coordinate.split(','))
      # print(x,y)
      px = (perspective_matrix[0][0]*x + perspective_matrix[0][1]*y + perspective_matrix[0][2]) / ((perspective_matrix[2][0]*x + perspective_matrix[2][1]*y + perspective_matrix[2][2]))
      py = (perspective_matrix[1][0]*x + perspective_matrix[1][1]*y + perspective_matrix[1][2]) / ((perspective_matrix[2][0]*x + perspective_matrix[2][1]*y + perspective_matrix[2][2]))
      transformed_coordinates.append((int(px), int(py)))

  transformed_coordinates
  # # Save the transformed coordinates to a new file
  with open('corners.txt', 'w') as file:
      for transformed_coordinate in transformed_coordinates:
          x, y = transformed_coordinate
          file.write(f'{int(x)},{int(y)}\n')

  cv2.imwrite('perspective_image.jpg', output_image)



In [7]:
A = rescale_image('plates/P28.jpg')
B = add_padding(A)
C = random_rotate(B)
C.save('semi.jpg')

D = perpective('semi.jpg')
# C.save('semi.jpg')


In [8]:
plot_image_with_corners('final/0.png','final/0.txt')

FileNotFoundError: [Errno 2] No such file or directory: 'final/0.png'

In [ ]:
# from PIL import Image, ImageDraw

# def crop_plate(image_path, vertices, output_path):
#     # Open the image
#     image = Image.open(image_path)

#     # Create a mask image
#     mask = Image.new('L', image.size, 255)
#     draw = ImageDraw.Draw(mask)

#     # Draw the shape using the provided vertices
#     draw.polygon(vertices, fill=255)

#     # Apply the mask to the image
#     masked_image = Image.new('RGBA', image.size)
#     masked_image.paste(image, mask=mask)

#     # Convert the image to RGB format
#     rgb_image = masked_image.convert('RGB')

#     # Crop the plate using the mask
#     plate = rgb_image.crop(rgb_image.getbbox())

#     # Save the cropped plate as a new file
#     plate.save(output_path)

# # Specify the path to the input image
# image_path = 'perspective_image.jpg'

# # Specify the coordinates for the vertices of the plate shape
# plate_vertices = [(366, 677), (296, 544),(316,502), (383, 660)]  # Replace x1, y1, x2, y2, x3, y3, x4, y4 with the actual values

# # Specify the path to save the cropped plate
# output_path = 'output.jpg'

# # Call the function to crop the plate and save it
# crop_plate(image_path, plate_vertices, output_path)


In [18]:
import PIL.Image
import random

def put_image_on_center(image1, image2):
  """Puts image1 on center image2 and replaces each pixel in image1 if is 0,0,0 with pixel from image2.

  Args:
    image1: The image to be placed on image2.
    image2: The image to be used as the background.

  Returns:
    The image with image1 placed on center image2.
  """

  image1 = PIL.Image.open(image1)
  image2 = PIL.Image.open(image2)

  # Get the width and height of the two images.
  width1, height1 = image1.size
  width2, height2 = image2.size
  print(width1, height1)
  print(width2, height2)
  # Calculate the center coordinates of image2.
  center_x = width2 // 2
  center_y = height2 // 2
  center_x = random.randint(width2//6,4*width2//6)
  center_y = random.randint(height2//6,4*height2//6)

  # Create a new image with the size of image2.
  new_image = PIL.Image.new("RGB", (width2, height2))

  # Paste image1 on center image2.
  new_image.paste(image1, (center_x - width1 // 2, center_y - height1 // 2))

  # # Iterate over the pixels in image1 and replace the black pixels with the pixels from image2.
  for x in range(width2):
    for y in range(height2):
      pixel = new_image.getpixel((x, y))
      if pixel[0]<20 and pixel[1]<20 and pixel[2]<20:
        new_image.putpixel((x, y), image2.getpixel((x, y)))

  with open('corners.txt', 'r') as file:
    coordinates = file.read().splitlines()

  with open('corners.txt', 'w') as file:
      for coordinate in coordinates:
          x, y = map(float, coordinate.split(','))
          file.write(f'{int(x+center_x - width1 // 2)},{int(y+center_y - height1 // 2)}\n')

  # shape_corners=[]
  # for coordinate in coordinates:
  #   x, y = map(float, coordinate.split(','))
  #   shape_corners.append((x,y))

  # for x in range(width2):
  #   for y in range(height2):
  #     if is_point_inside_shape((x,y),shape_corners)==0:
  #       new_image.putpixel((x, y), image2.getpixel((x, y)))

  return new_image

if __name__ == "__main__":
  image1_path = "perspective_image.jpg"
  image2_path = "backgrounds/B25.jpg"
  new_image = put_image_on_center(image1_path, image2_path)
  new_image.save("new_image2.png")


367 314
1360 680


In [19]:
# def is_point_inside_shape(point, shape_corners):
#     x, y = point
#     corners = shape_corners + [shape_corners[0]]  # Add the first corner to the end to close the shape

#     num_intersections = 0
#     for i in range(len(corners) - 1):
#         x1, y1 = corners[i]
#         x2, y2 = corners[i+1]

#         if ((y1 <= y < y2) or (y2 <= y < y1)) and (x < (x2 - x1) * (y - y1) / (y2 - y1) + x1):
#             num_intersections += 1

#     return num_intersections % 2 == 1

# # Example usage
# shape_corners = [(0, 0), (5, 0), (5, 5), (0, 5)]
# point = (2, 2)

# if is_point_inside_shape(point, shape_corners):
#     print("The point is inside the shape.")
# else:
#     print("The point is outside the shape.")


In [20]:
import random
random.random()

0.523806876264305

In [21]:
 
import os
import shutil
# os.mkdir('final')
plates= os.listdir('plates')
bkg=os.listdir('backgrounds')

index=0
for a in plates:
  for b in bkg:
    print(index)
    A = rescale_image('plates/'+a)
    B = add_padding(A)
    C = random_rotate(B)
    C.save('semi.jpg')

    # D = perpective('semi.jpg')
    new_image = put_image_on_center('semi.jpg', 'backgrounds/'+b)

    # new_image = put_image_on_center('perspective_image.jpg', 'backgrounds/'+b)
    new_image.save('final/'+str(index)+".png")
    shutil.copy('corners.txt','final/'+str(index)+'.txt')
    index+=1



0
386 346
748 499
1
336 260
900 600
2
600 498
1200 900
3
434 405
750 500
4
452 443
600 321
5
379 372
600 399
6
478 414
625 413
7
655 540
984 440
8
423 361
1024 536
9
670 670
1920 1200
10
547 389
640 442
11
478 459
768 512
12
550 380
850 460
13
429 354
620 413
14
647 671
259 194
15
487 347
1400 385
16
404 353
1600 1000
17
550 427
1200 900
18
636 662
907 509
19
321 288
1944 682
20
371 341
1920 1080
21
578 378
1024 768
22
619 528
4200 2800
23
482 327
1360 680
24
622 594
1360 680
25
353 224
1500 705
26
507 441
1632 1232
27
392 325
4677 3118
28
283 194
736 460
29
519 499
727 485
30
315 265
748 499
31
343 342
900 600
32
695 620
1200 900
33
594 428
750 500
34
563 460
600 321
35
590 473
600 399
36
432 410
625 413
37
306 266
984 440
38
301 265
1024 536
39
500 518
1920 1200
40
455 396
640 442
41
391 276
768 512
42
666 503
850 460
43
343 218
620 413
44
558 362
259 194
45
583 484
1400 385
46
418 409
1600 1000
47
678 526
1200 900
48
562 501
907 509
49
515 469
1944 682
50
337 267
1920 1080
51
630 51

In [ ]:
import time
while 1:
  time.sleep(100)
